In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, inspect
from dotenv import load_dotenv
import os

In [76]:
df = pd.read_csv('Data/Clean/cars_clean.csv')
apidf = pd.read_csv('Data/Clean/petroleum_clean.csv')

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8452 entries, 0 to 8451
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Year                   8452 non-null   int64  
 1   Make                   8452 non-null   object 
 2   Model                  8452 non-null   object 
 3   Used                   8452 non-null   bool   
 4   Price                  8452 non-null   int64  
 5   ConsumerRating         8452 non-null   float64
 6   ConsumerReviews        8452 non-null   int64  
 7   SellerType             8452 non-null   object 
 8   SellerName             8452 non-null   object 
 9   SellerRating           8452 non-null   float64
 10  SellerReviews          8452 non-null   int64  
 11  StreetName             8452 non-null   object 
 12  State                  8452 non-null   object 
 13  Zipcode                8452 non-null   object 
 14  DealType               8247 non-null   object 
 15  Comf

## Dimensions of cars dataset

### Car

In [78]:
# Dimensión: Vehículo con las nuevas columnas
car_dim = df[['Year', 'Make', 'Model', 'Drivetrain', 'MinMPG', 'MaxMPG', 
                         'FuelType', 'Transmission', 'Engine', 'ExteriorColor', 'InteriorColor', 
                         'Used', 'VIN', 'Stock#']].drop_duplicates().reset_index(drop=True)

# Asignar un ID único a cada vehículo
car_dim['ID_Car'] = car_dim.index + 1

# Mostrar la dimensión de vehículo
car_dim.head()

,Year,Make,Model,Drivetrain,MinMPG,MaxMPG,FuelType,Transmission,Engine,ExteriorColor,InteriorColor,Used,VIN,Stock#,ID_Car
0,2019,Toyota,Sienna SE,FWD,19,27,Gasoline,Automatic,3.5L V6 24V PDI DOHC,Red,Black,True,5TDXZ3DC2KS015402,22998646,1
1,2018,Ford,F-150 Lariat,4WD,19,24,Gasoline,Automatic,3.5L V6 24V PDI DOHC Twin Turbo,Shadow Black,Black,True,1FTEW1EG2JFD44217,22418A,2
2,2017,Ram,1500 Laramie,4WD,15,21,Gasoline,Automatic,5.7L V8 16V MPFI OHV,Granite Crystal Clearcoat Metallic,Black,True,1C6RR7VT5HS842283,NG277871G,3
3,2021,Honda,Accord Sport SE,FWD,29,35,Gasoline,CVT,1.5L I4 16V GDI DOHC Turbo,Gray,Â,True,1HGCV1F49MA038035,54237,4
4,2020,Lexus,RX 350,FWD,20,27,Gasoline,Automatic,3.5L V6 24V PDI DOHC,Eminent White Pearl,Birch,True,2T2AZMAA8LC156270,HDT4181A,5


### Seller

In [79]:
# Dimensión: Vendedor (sin cambios)
seller_dim = df[['SellerName', 'SellerType', 'State', 'Zipcode', 'StreetName']].drop_duplicates().reset_index(drop=True)

# Asignar un ID único a cada vendedor
seller_dim['ID_Seller'] = seller_dim.index + 1

# Mostrar la dimensión de vendedor
seller_dim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3972 entries, 0 to 3971
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   SellerName  3972 non-null   object
 1   SellerType  3972 non-null   object
 2   State       3972 non-null   object
 3   Zipcode     3972 non-null   object
 4   StreetName  3972 non-null   object
 5   ID_Seller   3972 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 186.3+ KB


## Ratings

In [80]:
# Dimensión: Ratings (nueva dimensión enfocada en calificaciones)
rating_dim = df[['ConsumerRating', 'SellerRating', 'ComfortRating', 'InteriorDesignRating', 
                        'PerformanceRating', 'ValueForMoneyRating', 'ExteriorStylingRating', 
                        'ReliabilityRating', 'DealType']].drop_duplicates().reset_index(drop=True)

# Asignar un ID único a cada set de ratings
rating_dim['ID_Rating'] = rating_dim.index + 1

# Mostrar la dimensión de ratings
rating_dim.head()

,ConsumerRating,SellerRating,ComfortRating,InteriorDesignRating,PerformanceRating,ValueForMoneyRating,ExteriorStylingRating,ReliabilityRating,DealType,ID_Rating
0,4.6,3.3,4.7,4.6,4.6,4.4,4.6,4.7,Great,1
1,4.8,4.8,4.9,4.8,4.8,4.6,4.8,4.7,Good,2
2,4.7,4.6,4.8,4.7,4.8,4.6,4.8,4.7,Good,3
3,5.0,4.6,4.9,5.0,4.9,5.0,5.0,5.0,NaN,4
4,4.8,4.8,4.9,4.8,4.8,4.7,4.8,4.9,Good,5


## Sells fact table

In [81]:
# Unir el dataframe original con la dimensión de vendedores para asignar el ID_Seller
df_hechos_vendedor = pd.merge(df, seller_dim, on=['SellerName', 'SellerType', 'State', 'Zipcode', 'StreetName'], how='left')

# Unir el dataframe original con la dimensión de vehículos para asignar el ID_Vehiculo
df_hechos_vehiculo = pd.merge(df, car_dim, on=['Year', 'Make', 'Model', 'Drivetrain', 'MinMPG', 'MaxMPG', 
                                                          'FuelType', 'Transmission', 'Engine' ,'ExteriorColor', 'InteriorColor', 'Used', 'VIN', 'Stock#'], how='left')

# Unir el dataframe original con la dimensión de ratings para asignar el ID_Rating
df_hechos_ratings = pd.merge(df, rating_dim, on=['ConsumerRating', 'SellerRating', 'ComfortRating', 
                                                        'InteriorDesignRating', 'PerformanceRating', 
                                                        'ValueForMoneyRating', 'ExteriorStylingRating', 
                                                        'ReliabilityRating', 'DealType'], how='left')

# Crear la tabla de hechos con solo las llaves foráneas y métricas
tabla_hechos = df_hechos_vendedor[['Price', 'Mileage', 'ConsumerReviews', 'SellerReviews', 'ID_Seller']].copy()

# Agregar el ID_Car desde df_hechos_vehiculo
tabla_hechos['ID_Car'] = df_hechos_vehiculo['ID_Car']

# Agregar el ID_Rating desde df_hechos_ratings
tabla_hechos['ID_Rating'] = df_hechos_ratings['ID_Rating']

# Asignar un ID único a cada venta (ID_Venta)
tabla_hechos['ID_Sell'] = df.index + 1

# Reordenar las columnas como en tu modelo dimensional
tabla_hechos = tabla_hechos[['ID_Sell', 'ID_Car', 'ID_Seller', 'ID_Rating', 'Price', 
                             'Mileage', 'ConsumerReviews', 'SellerReviews']]


In [90]:
tabla_hechos.head(3)

,ID_Sell,ID_Car,ID_Seller,ID_Rating,Price,Mileage,ConsumerReviews,SellerReviews
0,1,1,1,1,39998,29403,45,3
1,2,2,2,2,49985,32929,817,131
2,3,3,3,3,41860,23173,495,249


In [83]:
tabla_hechos['ID_Rating'].value_counts()

ID_Rating
256     55
705     37
501     33
202     22
209     22
        ..
5282     1
5283     1
5284     1
5285     1
6        1
Name: count, Length: 5285, dtype: int64

In [84]:
tabla_hechos['ID_Seller'].value_counts()

ID_Seller
717     31
19      31
183     27
179     23
479     17
        ..
3962     1
3961     1
3960     1
3959     1
3958     1
Name: count, Length: 3972, dtype: int64

In [85]:
rating_dim['ID_Rating'].value_counts()

ID_Rating
5285    1
1       1
2       1
3       1
4       1
       ..
10      1
9       1
8       1
7       1
6       1
Name: count, Length: 5285, dtype: int64

## Dimensions for petroleum dataset (API)

In [93]:
apidf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988 entries, 0 to 987
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   period              988 non-null    object 
 1   area-name           988 non-null    object 
 2   product             988 non-null    object 
 3   product-name        988 non-null    object 
 4   process             988 non-null    object 
 5   process-name        988 non-null    object 
 6   series-description  988 non-null    object 
 7   value($/GAL)        988 non-null    float64
 8   area                988 non-null    object 
dtypes: float64(1), object(8)
memory usage: 69.6+ KB


### Area

In [92]:
# Dimensión: area_dim
area_dim = apidf[['area', 'area-name']].drop_duplicates().reset_index(drop=True)

# Asignar un ID único a cada área
area_dim['area_ID'] = area_dim.index + 1

# Mostrar la dimensión de área
area_dim.head()

,area,area-name,area_ID
0,Region,Gulf Coast,1
1,Region,East Coast,2
2,Region,West Coast (except California),3
3,Region,Rocky Mountain,4
4,Region,East Coast (South),5


### Product

In [95]:
# Dimensión: product_dim
product_dim = apidf[['product', 'product-name']].drop_duplicates().reset_index(drop=True)

# Asignar un ID único a cada producto
product_dim['product_ID'] = product_dim.index + 1

# Mostrar la dimensión de producto
product_dim.head()

,product,product-name,product_ID
0,Gasoline,Total Gasoline,1
1,Diesel,No 2 Diesel,2
2,Gasoline,Conventional Regular Gasoline,3
3,Gasoline,Reformulated Regular Gasoline,4
4,Gasoline,Regular Gasoline,5


### Details

In [96]:
# Dimensión: details_dim
details_dim = apidf[['process', 'process-name', 'series-description']].drop_duplicates().reset_index(drop=True)

# Asignar un ID único a cada conjunto de detalles
details_dim['details_ID'] = details_dim.index + 1

# Mostrar la dimensión de detalles
details_dim.head()


,process,process-name,series-description,details_ID
0,PTE,Retail Sales,Gulf Coast All Grades All Formulations Retail ...,1
1,PTE,Retail Sales,East Coast No 2 Diesel Retail Prices (Dollars ...,2
2,PTE,Retail Sales,West Coast (PADD 5) Except California No 2 Die...,3
3,PTE,Retail Sales,Rocky Mountain Regular Conventional Retail Gas...,4
4,PTE,Retail Sales,New England (PADD 1A) Regular Conventional Ret...,5


### Fuel Fact

In [98]:
# Unir el dataframe original con la dimensión de área para asignar el area_ID
df_fuel_area = pd.merge(apidf, area_dim, on=['area', 'area-name'], how='left')

# Unir el dataframe original con la dimensión de producto para asignar el product_ID
df_fuel_product = pd.merge(apidf, product_dim, on=['product', 'product-name'], how='left')

# Unir el dataframe original con la dimensión de detalles para asignar el details_ID
df_fuel_details = pd.merge(apidf, details_dim, on=['process', 'process-name', 'series-description'], how='left')

# Crear la tabla de hechos con solo las llaves foráneas y las métricas
fuel_fact = df_fuel_area[['period', 'value($/GAL)']].copy()

# Agregar las llaves foráneas desde los dataframes de área, producto y detalles
fuel_fact['area_ID'] = df_fuel_area['area_ID']
fuel_fact['product_ID'] = df_fuel_product['product_ID']
fuel_fact['details_ID'] = df_fuel_details['details_ID']

# Asignar un ID único a cada registro en la tabla de hechos (fuel_ID)
fuel_fact['fuel_ID'] = fuel_fact.index + 1

# Reordenar las columnas como en tu modelo dimensional
fuel_fact = fuel_fact[['fuel_ID', 'period', 'area_ID', 'product_ID', 'details_ID', 'value($/GAL)']]

# Mostrar la tabla de hechos
fuel_fact.head()


,fuel_ID,period,area_ID,product_ID,details_ID,value($/GAL)
0,1,2024-09-30,1,1,1,2.793
1,2,2024-09-30,2,2,2,3.571
2,3,2024-09-30,3,2,3,3.797
3,4,2024-09-30,4,3,4,3.421
4,5,2024-09-30,5,3,5,3.058


## Save

In [87]:
# Guardar las tablas en archivos CSV
tabla_hechos.to_csv('Data/Dimensional_model/sells_fact.csv', index=False)
car_dim.to_csv('Data/Dimensional_model/car_dim.csv', index=False)
seller_dim.to_csv('Data/Dimensional_model/seller_dim.csv', index=False)
rating_dim.to_csv('Data/Dimensional_model/rating_dim.csv', index=False)

In [99]:
# Guardar las tablas en archivos CSV
fuel_fact.to_csv('Data/Dimensional_model/fuel_fact.csv', index=False)
area_dim.to_csv('Data/Dimensional_model/area_dim.csv', index=False)
product_dim.to_csv('Data/Dimensional_model/product_dim.csv', index=False)
details_dim.to_csv('Data/Dimensional_model/details_dim.csv', index=False)


Subimos a la base de datos

In [ ]:
load_dotenv()

# Conexión a la base de datos
localhost = os.getenv('LOCALHOST')
port = os.getenv('PORT')
nameDB = os.getenv('DB_NAME')
userDB = os.getenv('DB_USER')
passDB = os.getenv('DB_PASS')

engine = create_engine(f'postgresql+psycopg2://{userDB}:{passDB}@{localhost}:{port}/{nameDB}')

csv_directory = 'Data/Dimensional_model'

for csv_file in os.listdir(csv_directory):
    if csv_file.endswith('.csv'):
        table_name = csv_file.replace('.csv', '')
        
        location_file = os.path.join(csv_directory, csv_file)
        
        try:
            df = pd.read_csv(location_file, sep=",")
            
            df.to_sql(table_name, engine, if_exists='replace', index=False)
            
            print(f"Tabla '{table_name}' creada y datos subidos exitosamente.")
        
        except Exception as e:
            print(f"Error al subir los datos del archivo '{csv_file}': {e}")

        finally:
            engine.dispose()